In [2]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import re
import numpy as np
from selenium import webdriver
import time
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [3]:
def abrir_site():
    url = 'https://www.superabc.com.br/'
    driver = webdriver.Chrome(executable_path='C:/Users/pedro/selenium/chromedriver.exe')
    driver.maximize_window()
    driver.get(url)
    Select(driver.find_element_by_xpath('//*[@id="preHomeCidade"]')).select_by_value("Divinópolis")
    driver.get(url)
    elem = driver.find_element_by_xpath('//*[@id="popupCorona"]/span')
    elem.click()
    time.sleep(2)
    return driver

In [4]:
def carregar_mais():
    while True:
        try:
            wait = WebDriverWait(driver, 3)
            element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="carregar-mais"]')))
            element.click()
            time.sleep(2)
        except:
            break

In [2]:
import re
x = 'https://abcemcasa.vteximg.com.br/arquivos/ids/274822-1000-1000/7891962036212.jpg?v=637182402114630000'
ean = re.findall(r'(\d+)', re.findall(r'/\d+.jpg', x)[0])[0]
ean

'7891962036212'

In [5]:
def get_cat_I():
    driver = abrir_site()
    elem = driver.find_element_by_xpath('/html/body/div[2]/div/header/div/div[1]/div[2]/div[1]/button')
    elem.click()
    html = driver.page_source
    soup = BeautifulSoup(html)
    cat_raiz = soup.find_all('div', {'class':'menu-fixo-link'})
    cat_raiz = [item.find('a')['href'] for item in cat_raiz if len(item.find('a')['href'].split('/')) == 4]
    driver.quit()
    return cat_raiz

In [6]:
def get_cat_II(cat_I):
    driver = abrir_site()
    driver.get(cat_I)
    time.sleep(1)
    elem = driver.find_element_by_xpath('/html/body/div[2]/div/main/div[1]/div[2]/div/div/div[1]/button')
    elem.click()
    html = driver.page_source
    soup = BeautifulSoup(html)
    soup = soup.find_all('h4', {'class':'mercearia even'})
    links = [item.find('a')['href'] for item in soup]
    driver.quit()
    return links

In [7]:
def get_product_links(cat_II):
    driver = abrir_site()
    driver.get(cat_II)
    time.sleep(1)
    while True:
        try:
            wait = WebDriverWait(driver, 3)
            element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="carregar-mais"]')))
            element.click()
            time.sleep(2)
        except:
            break
    html = driver.page_source
    soup = BeautifulSoup(html)
    links = soup.find_all('a', {'class':'prateleira__name'})
    links = [item['href'] for item in links]
    driver.quit()
    return links

In [8]:
cat_I_lst = get_cat_I()

In [9]:
cat_I_lst

['http://www.superabc.com.br/mercearia',
 'http://www.superabc.com.br/bebidas',
 'http://www.superabc.com.br/carnes-e-pescados',
 'http://www.superabc.com.br/congelados-e-resfriados',
 'http://www.superabc.com.br/frios-e-laticinios',
 'http://www.superabc.com.br/padaria',
 'http://www.superabc.com.br/hortifruti',
 'http://www.superabc.com.br/higiene-e-beleza',
 'http://www.superabc.com.br/limpeza',
 'http://www.superabc.com.br/bazar',
 'http://www.superabc.com.br/pet-shop']

In [11]:
from multiprocess import Pool
pool = Pool(processes=4)
results = [pool.apply_async(get_cat_II, args=(x,)) for x in cat_I_lst]
pool.terminate()

In [8]:
driver = abrir_site()
driver.get('http://www.superabc.com.br/bebidas')
time.sleep(1)
elem = driver.find_element_by_xpath('/html/body/div[2]/div/main/div[1]/div[2]/div/div/div[1]/button')
elem.click()
html = driver.page_source
soup = BeautifulSoup(html)